In [1]:
import pandas as pd

In [2]:
df_orig = pd.read_csv('dataset/pn.csv.m3.120408.trim', sep='\t', header=None, names=['word', 'polarity', 'temp1'])

In [3]:
df_nega = df_orig.query('polarity=="n"')
df_nega

,word,polarity,temp1
46,２次感染,n,〜する（出来事）
47,２失点,n,〜する（出来事）
68,３連敗,n,〜する（出来事）
88,ＡＩＤＳ,n,〜である・になる（状態）客観
91,Ａ型肝炎,n,〜である・になる（状態）客観
...,...,...,...
13304,頽廃,n,〜する（出来事）
13305,顰蹙,n,〜する（行為）
13306,騙り,n,〜がある・高まる（存在・性質）
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観


In [4]:
# 読み仮名どうする
# 単純に読み仮名を振る → Pykaksi
# MeCabなら→読み仮名と発音を両方取得できる！
# MeCab使ってみるか


In [5]:
import MeCab

In [6]:
tagger = MeCab.Tagger()

In [7]:
print(tagger.parse("修学旅行に出かける"))

修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0
EOS



In [8]:
result = tagger.parse("修学旅行に出かける")
print(type(result))
print(result)

<class 'str'>
修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0
EOS



In [9]:
# EOSの手前を抽出
result = result.split('\nEOS')[0]

In [10]:
print(result)

修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0


In [11]:
[x.split('\t') for x in result.split('\n')]

[['修学', 'シューガク', 'シュウガク', '修学', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['旅行', 'リョコー', 'リョコウ', '旅行', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['に', 'ニ', 'ニ', 'に', '助詞-格助詞', '', '', ''],
 ['出かける', 'デカケル', 'デカケル', '出掛ける', '動詞-一般', '下一段-カ行', '終止形-一般', '0']]

In [12]:
''.join([x.split('\t')[1] for x in result.split('\n')])

'シューガクリョコーニデカケル'

In [13]:
def extract_yomi(word):
    tagger = MeCab.Tagger()
    result = tagger.parse(word)
    # EOSの手前を抽出
    result = result.split('\nEOS')[0]
    yomi = ''.join([x.split('\t')[1] for x in result.split('\n')])
    return yomi

In [14]:
extract_yomi("子供の落書き帳")

'コドモノラクガキチョー'

In [15]:
list(df_nega['word'])[1000:1010]

['猿知恵', '艶聞', '遠回り', '遠島', '遠慮気味', '鉛中毒', '塩害', '塩辛さ', '塩素臭', '汚さ']

In [16]:
lis = list(df_nega['word'])[1000:1010]

In [17]:
[extract_yomi(word) for word in lis]

['サルチエ',
 'エンブン',
 'トーマワリ',
 'エントー',
 'エンリョキミ',
 'ナマリチュードク',
 'エンガイ',
 'シオカラサ',
 'エンソシュー',
 'キタナサ']

In [18]:
lis = list(df_nega['word'])

In [19]:
df_nega['pronunciation'] = [extract_yomi(word) for word in lis]
df_nega

<ipython-input-19-baef14f1bba8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nega['pronunciation'] = [extract_yomi(word) for word in lis]


,word,polarity,temp1,pronunciation
46,２次感染,n,〜する（出来事）,２ジカンセン
47,２失点,n,〜する（出来事）,２シッテン
68,３連敗,n,〜する（出来事）,３レンパイ
88,ＡＩＤＳ,n,〜である・になる（状態）客観,エイズ
91,Ａ型肝炎,n,〜である・になる（状態）客観,Ａガタカンエン
...,...,...,...,...
13304,頽廃,n,〜する（出来事）,タイハイ
13305,顰蹙,n,〜する（行為）,ヒンシュク
13306,騙り,n,〜がある・高まる（存在・性質）,カタリ
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観,ヒニクノタン
